In [1]:
ID_MY_FILM = "302"

In [2]:
import os
import sys

os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [3]:
from pyspark import SparkContext
from pyspark import SparkConf

_________

In [4]:
conf = SparkConf().setAppName("roman_matiiv_lab_1")

In [5]:
sc = SparkContext(conf=conf)

In [6]:
!hdfs dfs -cat /labs/laba01/ml-100k/README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [7]:
# user id | item id | rating | timestamp. 
!hdfs dfs -head /labs/laba01/ml-100k/u.data | head -n 3

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116


In [8]:
rdd_data_row = sc.textFile("/labs/laba01/ml-100k/u.data")
rdd_data_row = rdd_data_row.repartition(3)

In [9]:
rdd_data_row.take(3)

['119\t392\t4\t886176814', '167\t486\t4\t892738452', '299\t144\t4\t877881320']

In [10]:
rdd_data = rdd_data_row.map(lambda row: row.split("\t"))

In [11]:
rdd_data.take(3)

[['119', '392', '4', '886176814'],
 ['167', '486', '4', '892738452'],
 ['299', '144', '4', '877881320']]

In [12]:
all_film_raiting = (rdd_data.map(lambda row: row[2])
                            .countByKey())

In [13]:
my_film_raiting = (rdd_data.filter(lambda row: row[1] == ID_MY_FILM)
                           .map(lambda row: row[2])
                           .countByKey())

In [16]:
result = {
           "hist_film": [  
              my_film_raiting['1'],
              my_film_raiting['2'],
              my_film_raiting['3'],
              my_film_raiting['4'],
              my_film_raiting['5']
           ],
           "hist_all": [  
              all_film_raiting['1'],
              all_film_raiting['2'],
              all_film_raiting['3'],
              all_film_raiting['4'],
              all_film_raiting['5']
           ]
        }

In [17]:
result_str = str(result)

In [18]:
result_str = result_str.replace("'",'"')

In [20]:
with open("/data/home/roman.matiiv/lab01.json", "w") as f:
    f.write(str(result_str))

In [21]:
sc.stop()